# imports

In [25]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import json
import shutil

# load model & origin dataset

In [26]:
TRAIN = True

In [27]:
def download_file(url, save_name):
    url = url
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)
    else:
        print('File already present, skipping download...')

In [28]:
if not os.path.exists('train'):
    !curl -L "https://public.roboflow.com/ds/xKLV14HbTF?key=aJzo7msVta" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

    dirs = ['train', 'valid', 'test']

    for i, dir_name in enumerate(dirs):
        all_image_names = sorted(os.listdir(f"{dir_name}/images/"))
        for j, image_name in enumerate(all_image_names):
            if (j % 2) == 0:
                file_name = image_name.split('.jpg')[0]
                os.remove(f"{dir_name}/images/{image_name}")
                os.remove(f"{dir_name}/labels/{file_name}.txt")

In [29]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git
%cd yolov5/
!/home/reedgern/anaconda3/envs/syntes_spb/bin/pip install -r requirements.txt

/home/reedgern/hacathons/sintez_spb/models/detector/yolov5


# load dataset

In [30]:
%cd ..

/home/reedgern/hacathons/sintez_spb/models/detector


In [31]:
DIR_DATASET = "bboxes"

In [32]:
!rm Dataset.zip
!curl -o Dataset.zip -L "https://drive.google.com/uc?export=download&confirm=yes&id=1zvRgqHn6dB5bEdD2oKeZOQu8Cx6Q-VwV"
!unzip -o Dataset.zip
!curl -o balls_markup.json -L "https://drive.google.com/uc?export=download&confirm=yes&id=1_OnXu63q_wImJVOlbui-Q4Gctsn_tMDG"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 62.6M  100 62.6M    0     0  3377k      0  0:00:19  0:00:19 --:--:-- 3479k
Archive:  Dataset.zip
  inflating: bboxes/.DS_Store        
  inflating: bboxes/00858b25-c5c3-4092-8ae7-5c61ada25097_frame_77.jpg  
  inflating: bboxes/04c68843-4119-4b46-8635-59e3a8ab60e3_frame_88.jpg  
  inflating: bboxes/06b6810b-7b6a-443e-970d-72a5df0d4dc7_frame_17.jpg  
  inflating: bboxes/09e5fd86-2cb1-4005-bfb4-1692a9160d7c_frame_313.jpg  
  inflating: bboxes/0a0699ee-cce2-4ca9-9038-ca6f0f1d9a79_frame_416.jpg  
  inflating: bboxes/0ae17d29-242b-40ec-bff6-fe6b23ac8824_frame_19.jpg  
  inflating: bboxes/0c901a44-8916-45e0-860c-874774412596_frame_59.jpg  
  inflating: bboxes/1934fd93-8315-401d-a534-71b4dfef602f_frame_2.jpg  
  inflating: bboxes/1fbcf03d-ad54-46e0-aebb-ff5

In [33]:
!mkdir -p train_ball/images
!mkdir -p val_ball/images
!mkdir -p train_ball/labels
!mkdir -p val_ball/labels

In [34]:
with open('balls_markup.json') as f:
  data = json.load(f)

video_keys = list(data["_via_image_id_list"])

In [35]:
def normalize_bbox_data(image_width, image_height, bbox):
    # Вычисляем ширину и высоту изображения
    image_width = float(image_width)
    image_height = float(image_height)

    # Извлекаем координаты границ bounding box
    x, y, w, h = bbox

    # Вычисляем нормализованные координаты границ bounding box
    x_normalized = x / image_width
    y_normalized = y / image_height
    w_normalized = w / image_width
    h_normalized = h / image_height

    # Возвращаем нормализованные координаты границ bounding box
    return [x_normalized, y_normalized, w_normalized, h_normalized]

In [36]:
from PIL import Image

boxes = []
dataset_ball = {}
for video_key in video_keys:
    file_name = data["_via_img_metadata"][video_key]["filename"]
    dataset_ball[file_name] = []
    for box in data["_via_img_metadata"][video_key]["regions"]:
      bbox = (box["shape_attributes"]['x'],
              box["shape_attributes"]['y'],
              box["shape_attributes"]['width'],
              box["shape_attributes"]['height'])
      file_img = DIR_DATASET + "/" + file_name
      im = Image.open(DIR_DATASET + "/" + file_name)
      image_width, image_height = im.size
      box = normalize_bbox_data(image_width, image_height, bbox)
      dataset_ball[file_name].append(box)

In [37]:
!rm train_ball/images/*
!rm train_ball/labels/*

new_class_indx = 5
for img in dataset_ball.keys():
  boxes = dataset_ball[img]
  if len(boxes) >= 1:
    file_img = DIR_DATASET + "/" + img
    if os.path.isfile(file_img):
      shutil.copyfile(file_img, f"train_ball/images/{img}")
    with open(f"train_ball/labels/{img[:-4]}.txt", "w") as text_file:
      for indx, box in enumerate(boxes):
          string_to_write = f"{new_class_indx}" + " " \
                          + str(box[0]) + " " \
                          + str(box[1]) + " " \
                          + str(box[2]) + " " \
                          + str(box[3])
          if indx + 1 != len(boxes):
            string_to_write += '\n'
          text_file.write(string_to_write)

In [38]:
!cp train/images/* train_ball/images

In [39]:
!cp train/labels/* train_ball/labels

In [40]:
!cp train_ball/images/* val_ball/images
!cp train_ball/labels/* val_ball/labels

In [41]:
import yaml

data = {
    'train': '../train_ball/images',
    'val': '../val_ball/images',
    'nc': 6,
    'names': ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck', 'GolfBall']
}

with open('ball_data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [42]:
%cd yolov5

/home/reedgern/hacathons/sintez_spb/models/detector/yolov5


In [43]:
from os import listdir

print("len images:", len(listdir("../train/images")), len(listdir("../train/labels")))
print("len new images:", len(listdir("../train_ball/images")), len(listdir("../train_ball/labels")))

len images: 439 439
len new images: 529 529


In [44]:
# check
files_images = "../" + "train_ball/images/"
files_labels = "../" + "train_ball/labels/"
files_count = 0
images_count = 0
for file_img in listdir(files_images):
  images_count += 1
  files_count += os.path.isfile(files_labels + file_img[:-4] + ".txt")
assert images_count == files_count, f"{files_count}, {images_count}"

In [45]:
test_file = list(dataset_ball.keys())[0]
with open(files_labels + test_file[:-4] + ".txt") as f:
    lines = f.readlines()
print(lines)
assert len(lines) == len(dataset_ball[test_file]), f"{len(lines)}, {len(dataset_ball[test_file])}"

['5 0.4861111111111111 0.6265625 0.018518518518518517 0.010416666666666666\n', '5 0.5842592592592593 0.5348958333333333 0.011111111111111112 0.007291666666666667']


In [46]:
# preprocess dataset, make uniform class distribution
for file_img in listdir(files_images):
  label_box = files_labels + file_img[:-4] + ".txt"
  with open(label_box) as f:
    lines = f.readlines()
  print(label_box)

../train_ball/labels/a37730f43629ed00_jpg.rf.f82eaff7cc2706caaaaa62b2431925ed.txt
../train_ball/labels/f1200c263667e455_jpg.rf.fnY9QjRYJ0Ed7j4H24ZC.txt
../train_ball/labels/d3825b2a6074c541_jpg.rf.c1bd59678beb04c3ed222062798eaac0.txt
../train_ball/labels/edf2794344b68506_jpg.rf.421da5c22eec803b8971abc5918e9b58.txt
../train_ball/labels/44e913fc3c81999f_jpg.rf.c4ca9965aa32ea5d6c4ec72ab66f57ca.txt
../train_ball/labels/33982df55de352d9_jpg.rf.7a51f29e9c77ce970ad26c00052c07cf.txt
../train_ball/labels/a7d33981079f545f_jpg.rf.XzJ6xvIOlomqkDUE59rE.txt
../train_ball/labels/c6976bd6-7622-4ed1-8d12-86b77ccb497a_frame_65.txt
../train_ball/labels/c350281af5ea9607_jpg.rf.f6Lu6ZXgVlpsww6P7Pvt.txt
../train_ball/labels/5152a73eea6c7a2e_jpg.rf.k5fkDuZlp8EqIfbWDOEy.txt
../train_ball/labels/33d9a8c7-dc35-42a3-a2b9-6003d6aa948c_frame_473.txt
../train_ball/labels/040d25a99e7b675b_jpg.rf.e1b3ced8dbbc7905d4fe5ac30622a419.txt
../train_ball/labels/d0fa518d2d1f9de2_jpg.rf.s4BvuNg1aeFu3G7MTcEk.txt
../train_ball/l

# train

In [97]:
TRAIN = True
EPOCHS = 90
RES_DIR = "custom_model"
if not os.path.exists(RES_DIR):
  os.makedirs(RES_DIR)

In [98]:
if TRAIN:
  !python train.py --data ../ball_data.yaml --weights yolov5s.pt \
    --img 640 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

train: weights=yolov5s.pt, cfg=, data=../ball_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=90, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=custom_model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7980MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv

       9/89      4.37G    0.05243    0.02161     0.0144          5        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.428      0.594      0.495      0.278

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      10/89      4.37G    0.05119    0.02109    0.01587          9        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.467      0.633       0.53      0.314

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      11/89      4.37G    0.04919    0.01892    0.01451          1        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066       0.48       0.48      0.479       0.27

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      12/8

      34/89      4.37G    0.03896    0.01726   0.006714          5        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.843      0.825      0.879      0.573

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      35/89      4.37G    0.03785    0.01632   0.006625          2        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.808      0.833      0.858      0.546

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      36/89      4.37G    0.03825    0.01724   0.008075          6        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.834      0.821      0.876      0.573

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      37/8

      59/89      4.37G    0.03282    0.01333   0.003918          3        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.921      0.903      0.925      0.675

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      60/89      4.37G     0.0333    0.01433   0.006346          6        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066       0.92      0.921      0.937      0.688

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      61/89      4.37G    0.03193     0.0147   0.005303          3        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.906      0.933      0.934      0.678

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      62/8

      84/89      4.37G    0.02641    0.01186   0.003056          1        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.938      0.934       0.94      0.744

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      85/89      4.37G    0.02666    0.01172   0.001973          1        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.939      0.938      0.946      0.751

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      86/89      4.37G    0.02711    0.01205   0.002076          8        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        529       1066      0.942      0.941      0.947      0.754

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      87/8

# inference

In [142]:
model_path = "runs/train/custom_model4"
video_to_test = "test_video_10"
path_to_video = f"../{video_to_test}.mp4"
folder_for_prediction = "inference_on_test_video"
result_video = f"../output_detection_{video_to_test}.mp4"


if not os.path.exists("inference_images"):
  os.makedirs("inference_images")

test_img = list(dataset_ball.keys())[0]

file_img = "../train_ball/images" + "/" + test_img
shutil.copyfile(file_img, f"inference_images/{test_img}")

# Function to show validation predictions saved during training.
def show_valid_results(RES_DIR):
    !ls runs/train/{RES_DIR}
    #EXP_PATH = f"runs/train/{RES_DIR}"
    EXP_PATH = f"{RES_DIR}"
    validation_pred_images = glob.glob(f"{EXP_PATH}/*_pred.jpg")
    print(validation_pred_images)
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

# Helper function for inference on images.
def inference(RES_DIR, data_path):
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f"Current number of inference detection directories: {infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    # Inference on images.
    !python detect.py --weights {RES_DIR}/weights/best.pt \
    --source {data_path} --name {INFER_DIR}
    return INFER_DIR

def visualize(INFER_DIR):
# Visualize inference images.
    INFER_PATH = f"runs/detect/{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*.jpg")
    print(infer_images)
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

In [93]:
# load test video
!curl -o ../test_video_1.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1UBuVZ3_7KzVGrZQ58RBxygHLyw6C8UPA"

#https://drive.google.com/file/d/1hTrXbRNhVjXCqWA2IvOijoIWeVHbs_Dv/view?usp=drive_link
!curl -o ../test_video_2.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1hTrXbRNhVjXCqWA2IvOijoIWeVHbs_Dv"

#https://drive.google.com/file/d/1cAbaUFYsiKmIystGckxAaLWGejbYFRd4/view?usp=drive_link
!curl -o ../test_video_3.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1cAbaUFYsiKmIystGckxAaLWGejbYFRd4"


#https://drive.google.com/file/d/14v69x5bXnlKebJ4jYDuWMqnyZ1My7u2q/view?usp=drive_link
!curl -o ../test_video_4.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=14v69x5bXnlKebJ4jYDuWMqnyZ1My7u2q"


#https://drive.google.com/file/d/1eeiu2eZqBJYYEPPRsu83yC1Znyk_Xumb/view?usp=drive_link
!curl -o ../test_video_5.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1eeiu2eZqBJYYEPPRsu83yC1Znyk_Xumb"


#https://drive.google.com/file/d/1ZO148XtAVQeaG-EhEChAsUAT5xu65k3c/view?usp=drive_link
!curl -o ../test_video_6.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=EhEChAsUAT5xu65k3c"


#https://drive.google.com/file/d/1GxpBH-yykhBHSiHLilqM3q-7iK0izTYX/view?usp=drive_link
!curl -o ../test_video_7.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1GxpBH-yykhBHSiHLilqM3q-7iK0izTYX"


#https://drive.google.com/file/d/1NH1X4CpoksWvgnWq9g7Q24KsRfa3BoZ-/view?usp=drive_link
!curl -o ../test_video_8.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1NH1X4CpoksWvgnWq9g7Q24KsRfa3BoZ-"

#https://drive.google.com/file/d/1CQhm5DbFIt6LY95263DTSEDb574fVPUP/view?usp=drive_link
!curl -o ../test_video_9.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1CQhm5DbFIt6LY95263DTSEDb574fVPUP"

#https://drive.google.com/file/d/1-1svnghiz8wpSRbEcObV5oufSnu-jZfY/view?usp=drive_link
!curl -o ../test_video_10.mp4 -L "https://drive.google.com/uc?export=download&confirm=yes&id=1-1svnghiz8wpSRbEcObV5oufSnu-jZfY"




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3587k  100 3587k    0     0  1215k      0  0:00:02  0:00:02 --:--:-- 3709k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1633k  100 1633k    0     0   990k      0  0:00:01  0:00:01 --:--:--  990k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2799k  100 2799k    0     0  1680k      0  0:00:01  0:00:01 --:--:-- 3993k
  % Total    % Received % Xferd  Average Speed   Tim

In [143]:
def getImageFrames(path):
  frames = []
  vidcap = cv2.VideoCapture(path)
  success, image = vidcap.read()
  count = 0
  print("success:", success)
  while success:
    frames.append(image)
    #cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
    success, image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1
  print("frames count:", count)
  return frames


if not os.path.exists(folder_for_prediction):
    os.makedirs(folder_for_prediction)
else:
  !rm folder_for_prediction/*
video_frames = getImageFrames(path_to_video)
for indx, frame in enumerate(video_frames):
  cv2.imwrite(f"{folder_for_prediction}/frame_{indx}.jpg", frame)
assert len(listdir(folder_for_prediction)) != 0

rm: cannot remove 'folder_for_prediction/*': No such file or directory
success: True
frames count: 843


In [144]:
IMAGE_INFER_DIR = inference(model_path, folder_for_prediction)

Current number of inference detection directories: 13
inference_14
detect: weights=['runs/train/custom_model4/weights/best.pt'], source=inference_on_test_video, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inference_14, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7980MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_0.jpg: 640x384 3 GolfBalls, 41.0ms
image 2/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_1.

image 55/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_147.jpg: 640x384 3 GolfBalls, 4.3ms
image 56/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_148.jpg: 640x384 3 GolfBalls, 4.2ms
image 57/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_149.jpg: 640x384 3 GolfBalls, 4.2ms
image 58/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_15.jpg: 640x384 3 GolfBalls, 4.2ms
image 59/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_150.jpg: 640x384 3 GolfBalls, 4.2ms
image 60/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_151.jpg: 640x384 3 GolfBalls, 4.3ms
image 61/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_152.jpg: 640x384 3 GolfBalls, 4.2ms
image 62/843 /home/reedgern/hacatho

image 115/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_200.jpg: 640x384 3 GolfBalls, 4.2ms
image 116/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_201.jpg: 640x384 3 GolfBalls, 4.3ms
image 117/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_202.jpg: 640x384 3 GolfBalls, 4.3ms
image 118/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_203.jpg: 640x384 3 GolfBalls, 4.2ms
image 119/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_204.jpg: 640x384 3 GolfBalls, 4.3ms
image 120/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_205.jpg: 640x384 3 GolfBalls, 4.3ms
image 121/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_206.jpg: 640x384 3 GolfBalls, 4.3ms
image 122/843 /home/reedger

image 174/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_254.jpg: 640x384 3 GolfBalls, 4.2ms
image 175/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_255.jpg: 640x384 3 GolfBalls, 4.3ms
image 176/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_256.jpg: 640x384 3 GolfBalls, 4.3ms
image 177/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_257.jpg: 640x384 3 GolfBalls, 4.2ms
image 178/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_258.jpg: 640x384 3 GolfBalls, 4.3ms
image 179/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_259.jpg: 640x384 3 GolfBalls, 4.2ms
image 180/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_26.jpg: 640x384 4 GolfBalls, 4.3ms
image 181/843 /home/reedgern

image 233/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_307.jpg: 640x384 3 GolfBalls, 4.3ms
image 234/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_308.jpg: 640x384 3 GolfBalls, 4.3ms
image 235/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_309.jpg: 640x384 3 GolfBalls, 4.2ms
image 236/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_31.jpg: 640x384 3 GolfBalls, 4.3ms
image 237/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_310.jpg: 640x384 3 GolfBalls, 4.2ms
image 238/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_311.jpg: 640x384 3 GolfBalls, 4.2ms
image 239/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_312.jpg: 640x384 3 GolfBalls, 4.2ms
image 240/843 /home/reedgern

...

image 587/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_626.jpg: 640x384 2 GolfBalls, 4.6ms
image 588/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_627.jpg: 640x384 2 GolfBalls, 4.5ms
image 589/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_628.jpg: 640x384 2 GolfBalls, 4.6ms
image 590/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_629.jpg: 640x384 2 GolfBalls, 4.2ms
image 591/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_63.jpg: 640x384 3 GolfBalls, 4.2ms
image 592/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_630.jpg: 640x384 2 GolfBalls, 4.2ms
image 593/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_631.jpg: 640x384 2 GolfBalls, 4.2ms
image 594/843 /home/reedgern

image 646/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_68.jpg: 640x384 3 GolfBalls, 4.2ms
image 647/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_680.jpg: 640x384 2 GolfBalls, 4.2ms
image 648/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_681.jpg: 640x384 2 GolfBalls, 4.2ms
image 649/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_682.jpg: 640x384 2 GolfBalls, 4.3ms
image 650/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_683.jpg: 640x384 2 GolfBalls, 4.2ms
image 651/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_684.jpg: 640x384 2 GolfBalls, 4.2ms
image 652/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_685.jpg: 640x384 2 GolfBalls, 4.2ms
image 653/843 /home/reedgern

image 705/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_732.jpg: 640x384 2 GolfBalls, 4.2ms
image 706/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_733.jpg: 640x384 2 GolfBalls, 4.2ms
image 707/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_734.jpg: 640x384 2 GolfBalls, 4.2ms
image 708/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_735.jpg: 640x384 2 GolfBalls, 4.2ms
image 709/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_736.jpg: 640x384 2 GolfBalls, 4.2ms
image 710/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_737.jpg: 640x384 2 GolfBalls, 4.2ms
image 711/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_738.jpg: 640x384 2 GolfBalls, 4.2ms
image 712/843 /home/reedger

image 764/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_786.jpg: 640x384 2 GolfBalls, 4.6ms
image 765/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_787.jpg: 640x384 2 GolfBalls, 4.8ms
image 766/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_788.jpg: 640x384 2 GolfBalls, 4.5ms
image 767/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_789.jpg: 640x384 2 GolfBalls, 4.3ms
image 768/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_79.jpg: 640x384 3 GolfBalls, 4.2ms
image 769/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_790.jpg: 640x384 2 GolfBalls, 4.2ms
image 770/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_791.jpg: 640x384 2 GolfBalls, 4.2ms
image 771/843 /home/reedgern

image 823/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_839.jpg: 640x384 2 GolfBalls, 4.3ms
image 824/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_84.jpg: 640x384 3 GolfBalls, 4.3ms
image 825/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_840.jpg: 640x384 2 GolfBalls, 4.2ms
image 826/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_841.jpg: 640x384 2 GolfBalls, 4.2ms
image 827/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_842.jpg: 640x384 2 GolfBalls, 4.3ms
image 828/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_85.jpg: 640x384 3 GolfBalls, 4.2ms
image 829/843 /home/reedgern/hacathons/sintez_spb/models/detector/yolov5/inference_on_test_video/frame_86.jpg: 640x384 3 GolfBalls, 4.2ms
image 830/843 /home/reedgern/h

In [145]:
inference_result_folder = "runs/detect/inference_14/"


import cv2
import os
import re

# Определение функции, которая извлекает число из имени файла
def extract_number(filename):
    match_ = re.search(r'\d+', filename)
    return int(match_.group())

def create_video_from_images(images_folder_path, output_video_path):
    image_files = os.listdir(images_folder_path)
    image_files = sorted(image_files, key=extract_number)

    img = cv2.imread(os.path.join(images_folder_path, image_files[0]))
    height, width, channels = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

    for image_file in image_files:
        img = cv2.imread(os.path.join(images_folder_path, image_file))
        video.write(img)

    cv2.destroyAllWindows()
    video.release()
    
create_video_from_images(inference_result_folder, result_video)